# COVID-19 Dashboard

Backgound

## 1. Introduction

## 2. Environment Setting

In [1]:
!pip install pandas

You should consider upgrading via the 'python -m pip install --upgrade pip' command.


## 3. Data Collection and Storage

### 3.1 Daily Data Collection
The nice thing is the government post data twice a day on the [Public Health Department](https://www.canada.ca/en/public-health/services/diseases/coronavirus-disease-covid-19.html). All I need to do is download the .csv file and update my own database. So first I need a crawler to download those file for me.

In [2]:
# This is crawler

### 3.2 Data Cleaning
Now, I have the .csv file in my computer, the next step is reading this file and see how it looks like. 

In [3]:
import pandas as pd
import numpy
import os

cwdpath=os.getcwd()
foldername='Sourse_Data'
filename=os.listdir(foldername)[-1]
covid=pd.read_csv(os.path.join(cwdpath,foldername,filename))
covid.head()

,pruid,prname,prnameFR,date,numconf,numprob,numdeaths,numtotal,numtested,numrecover,percentrecover,ratetested,numtoday,percentoday
0,35,Ontario,Ontario,31-01-2020,3,0,0.0,3,NaN,NaN,NaN,NaN,3.0,3.0
1,59,British Columbia,Colombie-Britannique,31-01-2020,1,0,0.0,1,NaN,NaN,NaN,NaN,1.0,1.0
2,1,Canada,Canada,31-01-2020,4,0,0.0,4,NaN,NaN,NaN,NaN,4.0,4.0
3,35,Ontario,Ontario,08-02-2020,3,0,0.0,3,NaN,NaN,NaN,NaN,0.0,0.0
4,59,British Columbia,Colombie-Britannique,08-02-2020,4,0,0.0,4,NaN,NaN,NaN,NaN,3.0,3.0


In [4]:
covid.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 702 entries, 0 to 701
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   pruid           702 non-null    int64  
 1   prname          702 non-null    object 
 2   prnameFR        702 non-null    object 
 3   date            702 non-null    object 
 4   numconf         702 non-null    int64  
 5   numprob         702 non-null    int64  
 6   numdeaths       689 non-null    float64
 7   numtotal        702 non-null    int64  
 8   numtested       628 non-null    float64
 9   numrecover      100 non-null    float64
 10  percentrecover  93 non-null     float64
 11  ratetested      0 non-null      float64
 12  numtoday        687 non-null    float64
 13  percentoday     687 non-null    float64
dtypes: float64(7), int64(4), object(3)
memory usage: 76.9+ KB


It shows that some columns like 'numrecover' and 'percentrecover' contains lots of null value, column 'ratetested' even has no data! Since in this project I don't need those columns so I decided to ignore them

In [5]:
covid=covid.iloc[:,[0,1,3,4,5,6,7,8]]
covid.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 702 entries, 0 to 701
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   pruid      702 non-null    int64  
 1   prname     702 non-null    object 
 2   date       702 non-null    object 
 3   numconf    702 non-null    int64  
 4   numprob    702 non-null    int64  
 5   numdeaths  689 non-null    float64
 6   numtotal   702 non-null    int64  
 7   numtested  628 non-null    float64
dtypes: float64(2), int64(4), object(2)
memory usage: 44.0+ KB


It looks great, next I need to change the data type of column 'date' to avoid problem when I store the data into my database.

In [6]:
covid.date=pd.to_datetime(covid.date,format='%d-%m-%Y')
covid.head()

,pruid,prname,date,numconf,numprob,numdeaths,numtotal,numtested
0,35,Ontario,2020-01-31,3,0,0.0,3,NaN
1,59,British Columbia,2020-01-31,1,0,0.0,1,NaN
2,1,Canada,2020-01-31,4,0,0.0,4,NaN
3,35,Ontario,2020-02-08,3,0,0.0,3,NaN
4,59,British Columbia,2020-02-08,4,0,0.0,4,NaN


The next step is dealing with Null value.

In [7]:
covid=covid.fillna(0).astype({'numdeaths':'int64','numtested':'int64'})
covid.head()

,pruid,prname,date,numconf,numprob,numdeaths,numtotal,numtested
0,35,Ontario,2020-01-31,3,0,0,3,0
1,59,British Columbia,2020-01-31,1,0,0,1,0
2,1,Canada,2020-01-31,4,0,0,4,0
3,35,Ontario,2020-02-08,3,0,0,3,0
4,59,British Columbia,2020-02-08,4,0,0,4,0


In [8]:
covid.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 702 entries, 0 to 701
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   pruid      702 non-null    int64         
 1   prname     702 non-null    object        
 2   date       702 non-null    datetime64[ns]
 3   numconf    702 non-null    int64         
 4   numprob    702 non-null    int64         
 5   numdeaths  702 non-null    int64         
 6   numtotal   702 non-null    int64         
 7   numtested  702 non-null    int64         
dtypes: datetime64[ns](1), int64(6), object(1)
memory usage: 44.0+ KB


### 3.3 Data Storage
Then I want to seperate the data into different parts because it makes more sense to store those data by province or places and order by date. It will be better to write a loop to do so.

In [9]:
# Build functions
def get_dataframe(framename):
    ft1=covid[covid['prname']==framename]
    ft2=ft1.iloc[:,5:].diff().rename(columns= {'numdeaths': 'newdeaths','numtotal': 'newtotal','numtested': 'newtested'}).fillna(0).astype('int64')
    return pd.concat([ft1,ft2],axis=1)

def savedb(dataframe, framename):
    dataframe.to_sql(framename,index=False,con=conn,if_exists='replace')
    print('%s data save into database' %(framename,))


In [10]:
import sqlite3

prlist=covid.prname.unique()
d={}
conn=sqlite3.connect('COVID19.db')
cur=conn.cursor()

for i in prlist:
    df=get_dataframe(i)
    d[i]=df
    savedb(df,i)
    conn.commit()



Ontario data save into database
British Columbia data save into database
Canada data save into database
Quebec data save into database
Alberta data save into database
Repatriated travellers data save into database
Saskatchewan data save into database
Manitoba data save into database
New Brunswick data save into database
Newfoundland and Labrador data save into database
Nova Scotia data save into database
Prince Edward Island data save into database
Yukon data save into database
Northwest Territories data save into database
Nunavut data save into database


In [11]:
conn.close()
print('done')

done


In [12]:
conn=sqlite3.connect('COVID19.db')
print('Database Open')
query='''SELECT date, newtotal FROM {}'''.format('Canada')
d=pd.read_sql_query(query,con=conn)
d

Database Open


,date,newtotal
0,2020-01-31 00:00:00,0
1,2020-02-08 00:00:00,3
2,2020-02-16 00:00:00,1
3,2020-02-21 00:00:00,1
4,2020-02-24 00:00:00,1
5,2020-02-25 00:00:00,1
6,2020-02-26 00:00:00,1
7,2020-02-27 00:00:00,1
8,2020-02-29 00:00:00,2
9,2020-03-01 00:00:00,9


In [13]:
import plotly.express as px
import plotly.graph_objects as go
fig=px.bar(d,x='date',y='newtotal')
fig.update_layout(xaxis=None,yaxis=dict(title='abc'))
fig.show()

# fig=go.Figure(data=[go.Bar(x='date',y='newtotal')])
# fig.update_layout(yaxis_title='abc')
# # fig.update_layout(yaxis=dict(title='abc'))
# fig.show()

In [14]:
from datetime import datetime as dt
dd=str(dt.today())
dd
# qurey = """SELECT numtotal FROM '{}' WHERE date='2020'"""

'2020-04-23 17:27:46.773132'

In [15]:
lst=['Ontario','Quebec','British Columbia']
del df
df=pd.DataFrame()
type(df)
for i in lst:
    q = """SELECT prname, numtotal FROM '{}' WHERE date=(SELECT max(date) FROM '{}')""".format(i,i)
    df=pd.concat([df,pd.read_sql_query(q,con=conn)],axis=0)
df

,prname,numtotal
0,Ontario,12245
0,Quebec,20965
0,British Columbia,1795


In [16]:
fig = px.bar(df, x=df.prname, y=df.numtotal)
fig.update_layout(title='Cases by Province',xaxis_title=None, yaxis_title=None)

In [17]:
fig=dict({
    'data':[{'type':'bar',
        'x':df.prname,
        'y':df.numtotal}],
})

import plotly.io as pio

pio.show(fig)


In [18]:
mode='Ontario'

print('New'+mode)

NewOntario


In [19]:
df = px.data.election()
geojson = px.data.election_geojson()
df

,district,Coderre,Bergeron,Joly,total,winner,result,district_id
0,101-Bois-de-Liesse,2481,1829,3024,7334,Joly,plurality,101
1,102-Cap-Saint-Jacques,2525,1163,2675,6363,Joly,plurality,102
2,11-Sault-au-Récollet,3348,2770,2532,8650,Coderre,plurality,11
3,111-Mile-End,1734,4782,2514,9030,Bergeron,majority,111
4,112-DeLorimier,1770,5933,3044,10747,Bergeron,majority,112
5,113-Jeanne-Mance,1455,3599,2316,7370,Bergeron,plurality,113
6,12-Saint-Sulpice,3252,2521,2543,8316,Coderre,plurality,12
7,121-La Pointe-aux-Prairies,5456,1760,3330,10546,Coderre,majority,121
8,122-Pointe-aux-Trembles,4734,1879,2852,9465,Coderre,majority,122
9,123-Rivière-des-Prairies,5737,958,1656,8351,Coderre,majority,123


In [26]:
qurey = """SELECT newtotal, newdeaths, newtested, numtotal, numdeaths, numtested FROM '{}' WHERE date=(SELECT max(date) FROM '{}')""".format(
        'Canada', 'Canada'
    )
df = pd.read_sql_query(qurey, con=conn)
df.at[0,'newtotal']

1768